In [1]:
# Import dependencies
import pandas as pd
import gmaps
import requests
# Import API key
from configure import g_key

In [2]:
# Read and load the cities database
city_data_df = pd.read_csv('weather_data\cities.csv')
# Get data type
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4) #[max(0,temp) for temp in max_temp])
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Heatmap of percentage of humidity
locations= city_data_df[['Lat', 'Lng']]
humidity = city_data_df['Humidity']
fig = gmaps.figure(center=(30.0, 31.0), zoom_level =1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the fig to plot data
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of cloudiness
locations = city_data_df[['Lat', 'Lng']]
cloudiness = city_data_df['Cloudiness']
fig = gmaps.figure(center=(30, 31), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=cloudiness, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of wind speed
locations = city_data_df[['Lat','Lng']]
wind = city_data_df['Wind Speed']
fig = gmaps.figure(center=(30,31), zoom_level=1.5)
heat_layer= gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4 )

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 93


In [9]:
# Filter dataset to find cities that fit the criteria
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & (city_data_df['Max Temp'] >= min_temp)]
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Kapaa,US,2019-11-08 04:55:03,22.08,-159.32,78.80,65,75,8.05
1,1,Vaini,IN,2019-11-08 04:55:03,15.34,74.49,75.20,78,40,0.76
4,4,Hithadhoo,MV,2019-11-08 04:55:03,-0.60,73.08,82.47,68,0,6.20
8,8,Butaritari,KI,2019-11-08 04:55:04,3.07,172.79,84.87,63,18,4.74
10,10,Pochutla,MX,2019-11-08 04:46:01,15.74,-96.47,77.00,78,40,6.93


In [10]:
preferred_cities_df.count()

City_ID       144
City          144
Country       144
Date          144
Lat           144
Lng           144
Max Temp      144
Humidity      144
Cloudiness    144
Wind Speed    144
dtype: int64

In [11]:
# Create dataframe called hotel_df to store hotel names along with city, country, max temperature and coordinates
hotel_df = preferred_cities_df[['City', 'Country', 'Max Temp', 'Lat', 'Lng']].copy()
hotel_df['Hotel Name'] = ''
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Kapaa,US,78.80,22.08,-159.32,
1,Vaini,IN,75.20,15.34,74.49,
4,Hithadhoo,MV,82.47,-0.60,73.08,
8,Butaritari,KI,84.87,3.07,172.79,
10,Pochutla,MX,77.00,15.74,-96.47,


In [12]:
# Set the parameters to search for a hotel
params = {'radius': 5000,
          'type': 'lodging',
          'key': g_key}

In [13]:
# Iterate through the Dataframe
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude
    lat = row['Lat']
    lng = row['Lng']
    # Add the latitude and longitude to the params dictionary as values to location key
    params['location'] = f'{lat},{lng}'
    # Use the search term: "lodging" and our latitude and longitude
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... skipping.')
        pass

    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Kapaa,US,78.80,22.08,-159.32,Courtyard by Marriott Kaua'i at Coconut Beach
1,Vaini,IN,75.20,15.34,74.49,Dandeli Lake County
4,Hithadhoo,MV,82.47,-0.60,73.08,Scoop Guest House
8,Butaritari,KI,84.87,3.07,172.79,Isles Sunset Lodge
10,Pochutla,MX,77.00,15.74,-96.47,Hotel Posada San Jose
14,Ambilobe,MG,90.73,-13.19,49.05,Hôtel & Restaurant DIANA Ambilobe
16,Ambikapur,IN,77.11,23.12,83.20,STARiHOTEL Ambikapur
17,Jizan,SA,80.60,16.89,42.56,Jazan Hotel
23,San Quintin,PH,86.29,17.54,120.52,San Quintin River Resort
33,Bambous Virieux,MU,77.00,-20.34,57.76,Casa Tia Villa


In [15]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [16]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [17]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))